In [2]:
# pip install pdfplumber
import pandas as pd
from pathlib import Path

In [3]:
#### data folder
data_folder = Path.cwd().parent / "Ofgem_Archetype"

# read the tables in the excel file
data = pd.read_excel(data_folder / "Ofgem energy consumer archetypes2024_Tables1.xlsx").dropna()

# display the data
data.head()


,Archetype,Number of\nhouseholds,ratio of households number,Average Annual Elec consumption (kWh),ratio of Average Annual Elec consumption,Average Annual Gas consumption (kWh),ratio of Average Annual Gas consumption,Total consumption (Elec+Gas) in KWh,Main heating Fuel,Gross\nannual\nhousehold\nincome,Attributes (key words)
0,A1,578333,0.021434,2742,0.025666,10933,0.054528,13675.0,Mains gas,15643.0,Lowest income; mains gas; retired; 75+ years o...
1,A2,868191,0.032176,2849,0.026668,9464,0.047202,12313.0,Mains gas,17327.0,Low income; housing association; single adults...
2,A3,883413,0.032740,3519,0.032939,10622,0.052977,14141.0,Mains gas,18195.0,Low income; mains gas; retired/unoccupied < 65...
3,B4,731318,0.027103,4811,0.045033,0,0.000000,4811.0,Electricity,18776.0,Low income; electric heating; retired/unoccupi...
4,B5,465288,0.017244,6597,0.061751,0,0.000000,6597.0,Electricity/Other (Solid fuel/LPG),22423.0,Low income; electric/solid fuel/LPG heating; 4...


### Calculate agents' share based on their fuel type and usage

In [ ]:
# data.to_csv(data_folder / "AgentShare.csv", index=False)

In [ ]:
# Add a function to account for partial matches in heating fuel names
def calculate_main_fuel_ratios(data, heating_fuel, consumption_column):
    # Filter rows where the heating fuel contains the given type (case-insensitive, exact match avoided)
    filtered_data = data[data['Main heating Fuel'].str.contains(heating_fuel, case=False, na=False, regex=False)].copy()
    
    # Calculate total consumption of the heating fuel for the group
    total_fuel_consumption = filtered_data[consumption_column].sum()
    
    # Calculate the ratio for each archetype within the group
    filtered_data['Fuel Consumption Ratio'] = (
        filtered_data[consumption_column] / total_fuel_consumption
    )
    
    # Select relevant columns for display
    return filtered_data[['Archetype', consumption_column, 'Fuel Consumption Ratio']]

# Calculate ratios for "Mains gas" based on gas consumption
mains_gas_fuel_ratios = calculate_main_fuel_ratios(data, 'Mains gas', 'Average Annual Gas consumption (kWh)')

# Calculate ratios for "Oil" based on electricity consumption (no specific Oil column in the dataset)
oil_fuel_ratios = calculate_main_fuel_ratios(data, 'Oil', 'Average Annual Elec consumption (kWh)')

# Calculate ratios for "Other (solid fuel/LPG)" based on electricity consumption (assuming lack of specific column)
other_fuel_ratios = calculate_main_fuel_ratios(data, 'Other (solid fuel/LPG)', 'Average Annual Elec consumption (kWh)')

# Calculate ratios for all main heating fuel types, including "Electricity"
electricity_fuel_ratios = calculate_main_fuel_ratios(data, 'Electricity', 'Average Annual Elec consumption (kWh)')



In [209]:
mains_gas_fuel_ratios

,Archetype,Average Annual Gas consumption (kWh),Fuel Consumption Ratio
0,A1,10933,0.054528
1,A2,9464,0.047202
2,A3,10622,0.052977
5,B6,10525,0.052493
6,C7,13119,0.065431
8,C9,13685,0.068254
9,D10,13981,0.06973
10,D11,8782,0.0438
11,D12,16065,0.080124
12,E13,16722,0.083401


In [210]:
oil_fuel_ratios

,Archetype,Average Annual Elec consumption (kWh),Fuel Consumption Ratio
16,G17,5901,0.321914
18,H19,4907,0.267689
23,J24,7523,0.410398


In [211]:
other_fuel_ratios

,Archetype,Average Annual Elec consumption (kWh),Fuel Consumption Ratio
4,B5,6597,0.370785
16,G17,5901,0.331666
17,G18,5294,0.297549


In [212]:
electricity_fuel_ratios

,Archetype,Average Annual Elec consumption (kWh),Fuel Consumption Ratio
3,B4,4811,0.170633
4,B5,6597,0.233978
7,C8,5587,0.198156
14,F15,6883,0.244121
15,F16,4317,0.153112


In [213]:
# Prepare a DataFrame with columns for archetypes and rows for fuel types
fuel_types = ['Mains gas', 'Oil', 'Other (solid fuel/LPG)', 'Electricity']

# Initialize an empty DataFrame with fuel types as index
ofgem_agent_df = pd.DataFrame(index=fuel_types)

# Add values for each fuel type into the DataFrame
def add_to_ofgem_agent_df(fuel_ratios, fuel_type, ofgem_agent_df):
    for _, row in fuel_ratios.iterrows():
        archetype = row['Archetype']
        ratio = row['Fuel Consumption Ratio']
        # Add the ratio for the specific fuel type and archetype
        ofgem_agent_df.loc[fuel_type, archetype] = ratio

# Add the calculated ratios to the DataFrame
add_to_ofgem_agent_df(mains_gas_fuel_ratios, 'Mains gas', ofgem_agent_df)
add_to_ofgem_agent_df(oil_fuel_ratios, 'Oil', ofgem_agent_df)
add_to_ofgem_agent_df(other_fuel_ratios, 'Other (solid fuel/LPG)', ofgem_agent_df)
add_to_ofgem_agent_df(electricity_fuel_ratios, 'Electricity', ofgem_agent_df)

# Fill missing values with 0 for archetypes not using certain fuels
ofgem_agent_df = ofgem_agent_df.fillna(0)


# Add a new row with values "new" under the header of ofgem_agent_df
# Create a DataFrame with the same columns, and a single row of "New"
new_row = pd.DataFrame([["new"] * ofgem_agent_df.shape[1]], columns=ofgem_agent_df.columns)

# Concatenate the new row above the existing ofgem_agent_df
ofgem_agent_df_with_unit_row = pd.concat([new_row, ofgem_agent_df])

ofgem_agent_df_with_unit_row



,A1,A2,A3,B6,C7,C9,D10,D11,D12,E13,...,J23,G17,H19,J24,B5,G18,B4,C8,F15,F16
0,new,new,new,new,new,new,new,new,new,new,...,new,new,new,new,new,new,new,new,new,new
Mains gas,0.054528,0.047202,0.052977,0.052493,0.065431,0.068254,0.06973,0.0438,0.080124,0.083401,...,0.081446,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Oil,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.321914,0.267689,0.410398,0.0,0.0,0.0,0.0,0.0,0.0
Other (solid fuel/LPG),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.331666,0.0,0.0,0.370785,0.297549,0.0,0.0,0.0,0.0
Electricity,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.233978,0.0,0.170633,0.198156,0.244121,0.153112


In [214]:
# Define a function to map fuel types to ofgem_agent_df index
def map_fuel_to_type(fuel):
    if fuel == "NGA":
        return "Mains gas"
    elif fuel in ["OIL", "HCO"]:
        return "Oil"
    elif fuel == "ELC":
        return "Electricity"
    elif fuel == "HYDROGEN" or fuel == "SOLAR": # for hot-water, some technologies/process use solar as fuel
        return None  # Exclude HYDROGEN
    elif fuel == "-":
        return 0 # the Unit row
    else:
        return "Other (solid fuel/LPG)"

In [215]:
def add_agents_to_technodata(techno_data, ofgem_agent_df):
    # Add a new column for the mapped fuel type
    techno_data['MappedFuelType'] = techno_data['Fuel'].apply(map_fuel_to_type)

    # Merge the two DataFrames
    technodata_add_agents = techno_data.merge(
        ofgem_agent_df_with_unit_row ,  # Use ofgem_agent_df with unit row
        left_on='MappedFuelType',  # Match on the mapped fuel type
        right_index=True,  # Use the index of ofgem_agent_df (fuel types)
        how='left'  # Keep all rows from techno_data
    )


    #There are NaN values in the archetype columns for rows where 'Fuel' is 'HYDROGEN'. Fill these with 0.

    # Identify rows where 'Fuel' is 'HYDROGEN' or 'SOLAR'
    hydrogen_rows = (technodata_add_agents['Fuel'] == 'HYDROGEN') | (technodata_add_agents['Fuel'] == 'SOLAR')


    # Fill NaN values in archetype columns for these rows with 0
    archetype_columns = [col for col in technodata_add_agents.columns if col in ofgem_agent_df.columns]
    technodata_add_agents.loc[hydrogen_rows, archetype_columns] =  (
        technodata_add_agents.loc[hydrogen_rows, archetype_columns]
        .fillna(0)
        .infer_objects(copy=False))


    # Remove all columns containing "Agent", "agent", or "MappedFuelType"
    columns_to_drop = [col for col in technodata_add_agents.columns 
                                if "Agent" in col or "agent" in col or col == "MappedFuelType"]
    # Drop the columns
    technodata_add_agents.drop(columns=columns_to_drop, inplace=True)

    return technodata_add_agents

In [216]:
MUSE_Files = "MUSE_Files"  # Define the MUSE_Files variable
subsectors = ["space_heat", "hot_water"]  # Define the space_heat variable

for sunsector in subsectors:
    technodata = pd.read_csv(Path.cwd().parent / MUSE_Files / sunsector / f"technodata_{sunsector}.csv").dropna()
    technodata_add_agents = add_agents_to_technodata(technodata, ofgem_agent_df)

    technodata_add_agents.to_csv(Path.cwd().parent / MUSE_Files / sunsector / f"technodata_{sunsector}_add_Ofgem_agents.csv", index=False)


C:\Users\jyang8\AppData\Local\Temp\ipykernel_6676\2123941575.py:24: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .fillna(0)
C:\Users\jyang8\AppData\Local\Temp\ipykernel_6676\2123941575.py:24: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .fillna(0)
